In [1]:
import os
import json
import time
import requests
import contextlib

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker

import pandas as pd
import numpy as np

import re
import urllib.parse

import matplotlib.pyplot as plt
import requests

In [2]:
from html import unescape
import unicodedata

In [3]:
def clean(text: str) -> str:
    text = text.strip()
    while True:
        prev_text = text
        text = unescape(text)
        if prev_text == text:
            break
    text = unicodedata.normalize("NFKC", text)
    return re.sub("\n\n\n+", "\n\n", re.sub("[ \t]+", " ", re.sub("\n[ \t]+", "\n", re.sub("\n\n+", "\n", re.sub("\r", "\n", text)))))

def strip_html(text: str) -> str:
    return re.sub(r"<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>", "", re.sub(r"<br\s*/?\s*>", "\n", text.strip()))

In [4]:
CONFIG_PATH = "config.json"
VERBOSE = False

In [5]:
CONFIG = None
ENGINES = {}
TABLES = {}
BINDS = {}
SESSION = None


def config_template():
    default_conn = {
        "dialect": "postgresql",
        "host": "localhost",
        "port": 5432,
        "dbname": "INVALID",
        "schema": "public",
        "user": "INVALID",
        "passwd": "INVALID",
    }
    return {
        "dbs": {
            "login": default_conn.copy(),
            "sm": default_conn.copy(),
            "exp": default_conn.copy(),
            "ap": default_conn.copy(),
            "blogs": default_conn.copy(),
        },
        "nlpapi": {
            "host": "localhost",
            "token": "INVALID",
            "write_access": "INVALID",
        },
    }


def get_config():
    global CONFIG
    
    if CONFIG is not None:
        return CONFIG
    if not os.path.exists(CONFIG_PATH):
        with open(CONFIG_PATH, "w") as fout:
            print(json.dumps(config_template(), indent=4, sort_keys=True), file=fout)
        raise ValueError(
            f"config file missing. new file was created at '{CONFIG_PATH}'. "
            "please correct values in file and run again")
    with open(CONFIG_PATH, "r") as fin:
        CONFIG = json.load(fin)
    config_out = {
        "dbs": {
            "login": CONFIG["dbs"]["login"].copy(),
            "sm": CONFIG["dbs"]["sm"].copy(),
            "exp": CONFIG["dbs"]["exp"].copy(),
            "ap": CONFIG["dbs"]["ap"].copy(),
            "blogs": CONFIG["dbs"]["blogs"].copy(),
        },
         "nlpapi": CONFIG["nlpapi"].copy(),
    }
    config_out["dbs"]["login"]["passwd"] = "*****"
    config_out["dbs"]["sm"]["passwd"] = "*****"
    config_out["dbs"]["exp"]["passwd"] = "*****"
    config_out["dbs"]["ap"]["passwd"] = "*****"
    config_out["nlpapi"]["token"] = "*****"
    config_out["nlpapi"]["write_access"] = "*****"
    print(f"loaded config\n{json.dumps(config_out, indent=2, sort_keys=True)}")
    return CONFIG


def get_engine(dbname):
    res = ENGINES.get(dbname)
    if res is not None:
        return res
    db = get_config()["dbs"][dbname]
    user = urllib.parse.quote_plus(db["user"])
    passwd = urllib.parse.quote_plus(db["passwd"])
    engine = sa.create_engine(
        f"{db['dialect']}://{user}:{passwd}@{db['host']}:{db['port']}/{db['dbname']}",
        echo=VERBOSE)
    engine = engine.execution_options(
        schema_translate_map={None: db['schema']})
    res = engine, sa.MetaData()
    ENGINES[dbname] = res
    return res


def get_table(dbname, tablename):
    global SESSION
    
    key = (dbname, tablename)
    res = TABLES.get(key)
    if res is not None:
        return res
    SESSION = None
    engine, metadata = get_engine(dbname)
    res = sa.Table(
        tablename,
        metadata,
        autoload_with=engine)
    TABLES[key] = res
    BINDS[res] = engine
    return res


@contextlib.contextmanager
def get_session():
    global SESSION
    
    session = SESSION
    if session is None:
        session = sessionmaker()
        session.configure(binds=BINDS)
        SESSION = session
    with session() as res:
        yield res

In [6]:
def call_nlpapi(path: str, payload: dict, *, is_write: bool, is_print: bool) -> dict:
    config = get_config()["nlpapi"]
    url = f"https://{config['host']}{path}"
    if is_print:
        print(url)
    res = requests.post(url, json={
        **payload,
        "token": config["token"],
        **({"write_access": config["write_access"]} if is_write else {}),
    }, timeout=120)
    res.raise_for_status()
    return res.json()

In [7]:
# global tables
t_tags = get_table("login", "tags")
t_users = get_table("login", "users")

# solution mapping tables
t_sm_pads = get_table("sm", "pads")
t_sm_tagging = get_table("sm", "tagging")

# action plan tables
t_ap_pads = get_table("ap", "pads")
t_ap_tagging = get_table("ap", "tagging")

# experiments tables
t_exp_pads = get_table("exp", "pads")
t_exp_tagging = get_table("exp", "tagging")

loaded config
{
  "dbs": {
    "ap": {
      "dbname": "action_plans_platform",
      "dialect": "postgresql",
      "host": "acclabs-global-login.postgres.database.azure.com",
      "passwd": "*****",
      "port": 5432,
      "schema": "public",
      "user": "acclabshqadmin@acclabs-global-login"
    },
    "blogs": {
      "dbname": "blogs",
      "dialect": "postgresql",
      "host": "acclabs.postgres.database.azure.com",
      "passwd": "acclabsblogs@2023",
      "port": 5432,
      "schema": "public",
      "user": "undpacclab@acclabs"
    },
    "exp": {
      "dbname": "experiments_platform",
      "dialect": "postgresql",
      "host": "acclabs-global-login.postgres.database.azure.com",
      "passwd": "*****",
      "port": 5432,
      "schema": "public",
      "user": "acclabshqadmin@acclabs-global-login"
    },
    "login": {
      "dbname": "postgres",
      "dialect": "postgresql",
      "host": "acclabs-global-login.postgres.database.azure.com",
      "passwd": "*****",

/var/folders/ff/zrdw3xx56dd1w9xbl6x_yl0c0000gn/T/ipykernel_24204/977369557.py:93: SAWarning: Did not recognize type 'ltree' of column 'version'
  res = sa.Table(


In [8]:
call_nlpapi("/api/query_embed", {
    "input": "africa",
    "offset": 0,
    "limit": 10,
    "db": "test",
    "score_threshold": 0.2,
    "filters": {"iso3": ["egy"]},
}, is_write=False, is_print=True)

https://acclabs-nlpapi.azurewebsites.net/api/query_embed


{'hits': [], 'status': 'ok'}

In [9]:
tags = {}
with get_session() as session:
    stmt = sa.select(t_tags.c.id, t_tags.c.name, t_tags.c.type)
    for row in session.execute(stmt):
        tags[f"{row[2]}-{row[0]}"] = row[1]

In [10]:
status_map = {
    2: "preview",
    3: "public",
}

In [17]:
skip_ahead = 4989

In [ ]:
with get_session() as session:
    destination = "main"  # "test"
    names = ["solution", "actionplan", "experiment"]
    urls = [
        "https://solutions.sdg-innovation-commons.org/en/view/pad?id=",
        "https://learningplans.sdg-innovation-commons.org/en/view/pad?id=",
        "https://experiments.sdg-innovation-commons.org/en/view/pad?id=",
    ]
    dbs = [
        t_sm_pads,
        t_ap_pads,
        t_exp_pads,
    ]
    doc_types = [
        "solution",
        "action plan",
        "experiment",
    ]    
    count = 0
    start_total_time = time.monotonic()
    try:
        for (name, url_prefix, pad_db, doc_type) in zip(names, urls, dbs, doc_types):
            stmt = sa.select(pad_db.c.status, pad_db.c.full_text, pad_db.c.id, pad_db.c.update_at, pad_db.c.title)
            stmt = stmt.where(pad_db.c.status > 1)
            start_time = time.monotonic()
            time_count = 0
            for row in session.execute(stmt):
                if count < skip_ahead:
                    count += 1
                    continue
                status = status_map[int(row[0])]
                if count % 100 == 0:
                    print(f"adding #{count} {name} {row[2]} {status} {row[3]}")
                url = f"{url_prefix}{int(row[2])}"
                title = f"{row[4]}".strip()
                if not title:
                    title = url
                call_nlpapi("/api/add_embed", {
                    "input": f"{row[1]}",
                    "base": f"{name}",
                    "doc_id": int(row[2]),
                    "url": url,
                    "title": title,
                    "meta": {
                        "status": status,
                        "date": f"{row[3]}",
                        "doc_type": doc_type,
                    },
                    "db": destination,
                }, is_write=True, is_print=count % 100 == 0)
                time_count += 1
                if count % 100 == 0:
                    print(
                        f"took avg {(time.monotonic() - start_time) / time_count if time_count > 0 else '?'}s for {time_count}")
                    first = False
                    start_time = time.monotonic()
                    time_count = 0
                count += 1
    finally:
        duration = time.monotonic() - start_total_time
        print(f"processed: {count} in {duration}s avg time {duration / (count - skip_ahead) if (count - skip_ahead) > 0 else '?'}s")

adding #5000 experiment 249 preview 2022-04-29 10:30:51.062276+00:00
https://acclabs-nlpapi.azurewebsites.net/api/add_embed
took avg 3.52529607982918s for 12
adding #5100 experiment 340 preview 2023-02-27 17:06:14.854987+00:00
https://acclabs-nlpapi.azurewebsites.net/api/add_embed
took avg 2.8541509579098783s for 100
adding #5200 experiment 479 preview 2024-03-07 15:16:15.420727+00:00
https://acclabs-nlpapi.azurewebsites.net/api/add_embed
took avg 3.0074882820795756s for 100


In [ ]:
t_blogs = get_table("blogs", "articles")
t_links = get_table("blogs", "links")